In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
# Takes quite a bit of time for date-inference
# Optimization: Consider manual caching in a dict (top StOvflw answer)
data = pd.read_csv("PecanStreet_Project/dataport-export_gas_oct2015-mar2016.csv")

## Question 1

### Q1.1

How many houses are included in the measurement study? Are there any malfunctioning meters? If so, identify them and the time periods where they were malfunctioning. The information below regarding data collection may be useful.

In [ ]:
print("Number of unique houses: ", len(data["dataid"].unique()))

In [ ]:
# Calculating Time Difference in Seconds 
# Caclulating Difference between readings (in order)
# Assumption: localminute column is clean and error free
merged_df = pd.DataFrame()
for k,df in data.groupby(["dataid"]): 
    df.sort_values(by=["localminute"], inplace=True)
    df["val_diff"] = df["meter_value"].diff()
    df["time_diff"] = pd.to_datetime(df['localminute'], utc=True, infer_datetime_format=True, cache=True).diff().astype('timedelta64[s]')
    merged_df = merged_df.append(df)


### Q1.2

Generate hourly readings from the raw data. Select one month from the 6-month study interval and plot the hourly readings (time-series) for that month. Hint: You will have to decide what to do if there are no readings for a certain hour.

In [ ]:
# How many readings do I have per house?  
keys, counts = zip(*[(k, len(df)) for k, df in merged_df.groupby("dataid")])
px.scatter(x=keys, y=counts, log_y=True, color=counts, title="Number of Readings per House").show()

In [ ]:
# looks like some houses don't have a lot of readings. 
# Let's extract the house_ids
MIN_NUM_READING_THRESHOLD = 50
less_readings = [(x, y) for x, y in list(zip(keys, counts)) if y < MIN_NUM_READING_THRESHOLD]
less_readings

In [ ]:
# Q: get rid of these houses in less_readings before next cells?

In [ ]:
keys, means, stds, maxs, mins = zip(*[(k, df["val_diff"].mean(), df["val_diff"].std(), df["val_diff"].max(), df["val_diff"].min()) for k, df in merged_df.groupby("dataid")])

In [ ]:
px.scatter(x=keys, y=means, log_y=True, title="Mean Meter Diff per House").show()

In [ ]:
px.scatter(x=keys, y=maxs, log_y=True, title="Max Meter Diff per House").show()

In [ ]:
px.scatter(x=keys, y=mins, title="Min Meter Diff per House").show()

In [ ]:
# Interesting. Some houses haev negative diff. Malfunctioning meters? 
# How many bad readings and how many houses?
negative_diff_df = merged_df[merged_df['val_diff'] < 0]
print(len(negative_diff_df))                # how many readings
print(negative_diff_df['dataid'].nunique()) # how many houses
print(negative_diff_df['dataid'].unique())  # which houses

In [ ]:
# When do the spikes happen for each house? 
# Note that this diff < 0 signifies represents the RIGHT END of a spike, _NOT_ PEAK.
spike_dict = {} # dataid: number of spikes, first_spike_time, last_spike_time, diff_first_last_in_hours
for k, df in negative_diff_df.groupby('dataid'): 
    num_spikes = len(df)
    first_spike_end = df['localminute'].iloc[0]
    last_spike_end = df['localminute'].iloc[-1]
    spike_dict[k] = (num_spikes, first_spike_end, last_spike_end, (pd.to_datetime(last_spike_end, utc=True)-pd.to_datetime(first_spike_end, utc=True)))
spike_dict

In [ ]:
# Looks like it happens throughout the time period we have
# TODO: need to create a list for each house for when do the spikes happen? 
px.scatter(negative_diff_df, x='localminute')

In [ ]:
keys, means, stds, maxs, mins = zip(*[(k, df["time_diff"].mean(), df["time_diff"].std(), df["time_diff"].max(), df["time_diff"].min()) for k, df in merged_df.groupby("dataid")])

px.scatter(x=keys, y=means, log_y=True, title="Mean Time between readings (s) per House").show()
px.scatter(x=keys, y=mins, title="Minimum Time between readings (s) per House").show()
px.scatter(x=keys, y=maxs, log_y=True, title="Max Time between readings (s) Per House").show()
px.scatter(x=keys, y=stds, log_y=True, title="Standard Deviation of Time between (s) readings by House").show()

In [ ]:
# Resampling per hour

resampled_df = pd.DataFrame()

for k, df in merged_df.groupby("dataid"): 
    df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df['localminute'], utc=True, infer_datetime_format=True, cache=True)))
    df.drop(columns=["localminute"], inplace=True)

    df.drop(columns=["val_diff", "time_diff"], inplace=True)

    # keep last sample of every hour only
    # missing hours become NA
    sample = df.resample('1h').last()

    # fix data_id for missing rows
    sample["dataid"].fillna(k, inplace=True)

    # TODO configurable hyperparameter for "fixing data"
    sample["meter_value"] = sample["meter_value"].interpolate()

    resampled_df = resampled_df.append(sample)

In [ ]:
resampled_december_data = resampled_df.loc['2015-12-01':'2015-12-31']
px.line(resampled_december_data, x=resampled_december_data.index, y="meter_value", color="dataid").show()

In [ ]:
# Question: How do we clkean the negative readings? 
#     Option 1: Modify merged_df so that diff = 0 
#     Option 2: Get rid of the rows in merged_df so our fill can path it automatically (OPTION CHOSEN IN CODE BELOW)

spikeless_resampled_df = pd.DataFrame()

for k, df in merged_df.groupby("dataid"): 
    df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df['localminute'], utc=True, infer_datetime_format=True, cache=True)))
    df.drop(columns=["localminute"], inplace=True)

    spikeless_df = df[~(df['val_diff'].shift(-1) < 0)] 
    spikeless_df['val_diff'] = spikeless_df['meter_value'].diff()


    # Need to do this because some spikes are less "sharp" than 1 timestemp
    for i in range(10):  # by right should be doing UNTIL no more spikes left. Tested to see no more spikes after 10 passes
        spikeless_df = spikeless_df[~(spikeless_df['val_diff'].shift(-1) < 0)] 
        spikeless_df['val_diff'] = spikeless_df['meter_value'].diff()

    spikeless_df.drop(columns=["val_diff", "time_diff"], inplace=True)

    spikeless_sample = spikeless_df.resample('1h').last()

    spikeless_sample["dataid"].fillna(k, inplace=True)

    spikeless_sample["meter_value"] = spikeless_sample["meter_value"].interpolate()

    spikeless_resampled_df = spikeless_resampled_df.append(spikeless_sample)

In [ ]:
# let's see how spikeless looks now. 
spikeless_resampled_december_data = spikeless_resampled_df.loc['2015-12-01':'2015-12-31']
px.line(spikeless_resampled_december_data, x=spikeless_resampled_december_data.index, y="meter_value", color="dataid").show()

In [ ]:
px.line(spikeless_resampled_df, x=spikeless_resampled_df.index, y="meter_value", color="dataid").show()

In [ ]:
# Correlation Running WITH spikes
grouped_df = resampled_df.groupby("dataid")
top5_pair = {}
results = {}
for k1, df_1 in grouped_df: 
    correlations = []
    for k2, df_2 in grouped_df: 
        if k1==k2: 
            continue 
        # Does this align data? If not, gotta manually align first. 
        # What about misaligned data? (eg: meter A started from Oct 10th, while meter B started from Oct 1st. 
        # Same concern about the last reading)
        correlation = df_1["meter_value"].corr(df_2["meter_value"])  
        correlations.append((correlation, k2))
    correlations.sort(reverse=True)
    results[k1] = correlations[:5]

In [ ]:
# for item in results.items(): 
#     print(item)

In [ ]:
# Correlation run WITHOUT spikes

grouped_df = spikeless_resampled_df.groupby("dataid")

results = {}
for k1, df_1 in grouped_df: 
    correlations = []
    for k2, df_2 in grouped_df: 
        if k1==k2: 
            continue 
        # Does this align data? If not, gotta manually align first. 
        # What about misaligned data? (eg: meter A started from Oct 10th, while meter B started from Oct 1st. 
        # Same concern about the last reading)
        correlation = df_1["meter_value"].corr(df_2["meter_value"])  
        correlations.append((correlation, k2))
    correlations.sort(reverse=True)
    results[k1] = correlations[:5]


### Q1.3

Intuitively, we expect that gas consumption from different homes to be correlated. For example, many homes would experience higher consumption levels in the evening when meals are cooked. For each home, find the top five homes with which it shows the highest correlation.

### Q1.3 Corr matrix without spikes, network graph, heatmap

In [ ]:
for item in results.items(): 
    print(item)

In [ ]:
# Correlation run WITHOUT spikes and with align

grouped_df = spikeless_resampled_df.groupby("dataid")

results = {}
for k1, df_1 in grouped_df: 
    correlations = []
    for k2, df_2 in grouped_df: 
        if k1==k2: 
            continue 
        # Does this align data? If not, gotta manually align first. 
        # What about misaligned data? (eg: meter A started from Oct 10th, while meter B started from Oct 1st. 
        # Same concern about the last reading)
        df_1, df_2 = df_1.align(df_2, join='inner', axis=0) #Inner join to align data before calculating corr
        correlation = df_1["meter_value"].corr(df_2["meter_value"], method='pearson')  
        correlations.append((correlation, k2))
    correlations.sort(reverse=True)
    results[k1] = correlations[:5]

In [ ]:
## default, no method specified
results

In [ ]:
# Convert dict into df corr matrix
corr_df = pd.DataFrame()
for key, val in results.items():
    for res in val:
        corr_df.at[key, res[1]] = res[0]

In [ ]:
corr_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(30, 30))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
# cmap="YlGnBu" #alternative color map
sns.heatmap(corr_df, cmap=cmap, vmax=.3, center=0, xticklabels=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .7})

###### plot using matlab (even uglier though)
# f = plt.figure(figsize=(19, 15))
# plt.matshow(corr_df, fignum=f.number)
# plt.xticks(range(corr_df.shape[1]), corr_df.columns, fontsize=14, rotation=45)
# plt.yticks(range(corr_df.shape[1]), corr_df.columns, fontsize=14)
# cb = plt.colorbar()
# cb.ax.tick_params(labelsize=14)
# plt.title('Correlation Matrix', fontsize=16);

In [ ]:
### Use builtin heatmap plotter from pandas
corr_df.style.background_gradient(cmap='coolwarm')

In [ ]:
### Plot network graph source:https://python-graph-gallery.com/327-network-from-correlation-matrix/
import networkx as nx
# Transform it in a links data frame (3 columns only):
links = corr_df.stack().reset_index()
links.columns = ['var1', 'var2','value']
links
 
# Keep only correlation over a threshold and remove self correlation (cor(A,A)=1)
links_filtered=links.loc[ (links['value'] > 0.8) & (links['var1'] != links['var2']) ]
links_filtered

# Build your graph
G=nx.from_pandas_edgelist(links_filtered, 'var1', 'var2')
# Plot the network:
nx.draw(G, with_labels=True,
        node_color='cyan',
        node_size=100,
        edge_color='black',
        linewidths=0.05, font_size=5)

In [ ]:
#with align
for item in results.items(): 
    print(item)